#### **we first starting by importing the loader**

In [ ]:
from kloppy import pff #import the loader which is commonly the name of the data provider.

##### **Path to the necessaries files to load with kolppy, the final match (game_id=10517) of the 2022 world cup.**

In [4]:
roster = "/home/student/Documents/AIMS/Intership/pysport/first_week/data/10517_roster.json"
metadata = "/home/student/Documents/AIMS/Intership/pysport/first_week/data/10517_metadata.json"
raw_data = "/home/student/Documents/AIMS/Intership/pysport/first_week/data/pff_10517.jsonl.bz2"

##### **using the path variables and the load_tracking() function of the pff loader, we load the game data.**

In [ ]:
dataset = pff.load_tracking(
    meta_data= metadata,
    roster_meta_data= roster,
    raw_data = raw_data,
    # Optional Parameters
    coordinates = "pff", # here we specify that we are going to use the pff pitch size and  orientation
    sample_rate = None, # we keep the number of laded frame as given by the provider number of frame per second ()
    limit = None # We load the total frames available wich are around (5400 seconds * 25 frames per second)
)

In [67]:
# let take a look of how many frames it contains
print(f"Te dataset contains {len(dataset)} frames. ")
# we first start by cheking the approximative number of minute recorded
def what_is_this(frame_rate=25):
    total_recorded_time  = len(dataset)/frame_rate
    time_in_min = total_recorded_time/60 # to get the result in minutes
    print(f"This is the record of around {time_in_min:0.2f} min of the 2022 football final world cup ")

what_is_this()

Te dataset contains 130308 frames. 
This is the record of around 86.87 min of the 2022 football final world cup 


##### **Strange... it seems like the number of frames doesn't match with our excpectation**

##### **Assumptions**
- the number of frame is not 25 per second but less than this.
- the provider doesn't provide record of the entire game.

In [40]:
# let go into attribute of the data_loaded
def of_type(data):
    return data.dataset_type.name

print(f"This is a {of_type(dataset)} dataset.")

This is a TRACKING dataset.


##### **implement some functions available on Tracking Dataset to navigate within the data**

In [68]:
# exploration of metadata variabes
dataset.metadata?

Type:        Metadata
String form: Metadata(periods=[Period(id=1, start_timestamp=datetime.timedelta(seconds=154, microseconds=48782 <...> .utc), game_week=8, game_id=10517, home_coach=None, away_coach=None, officials=[], attributes={})
File:        ~/miniconda3/envs/pysport_env/lib/python3.11/site-packages/kloppy/domain/models/common.py
Docstring:  
Metadata for a dataset.

Metadata is additional information about the dataset that is not part of
the actual data. It includes information about the teams, the pitch
dimensions, the orientation, the provider, and more.

Attributes:
    game_id: Game id of the game from the provider.
    date: Date of the game.
    game_week: Game week (or match day) of the game. It can also be the stage
        (ex: "8th Finals"), if the game is happening during a cup or a play-off.
    periods: List of [`Period`][kloppy.domain.models.common.Period] entities.
    teams: `[home_team, away_team]`.
    coordinate_system: The coordinate system in which the dat

In [ ]:
# this give us some variable to handle informations about the game store in metadata 
# here we see that we can access the game_id, the date were the match was play the stage of the competion or the game_week
# the informations about the teams, the final score and important ploting information like the pitch dimension the attacking orientation and the cordinate system.
# Also we have extra information like the provider the frame_rate and the periods. 

In [97]:
# Let remember when was the 2022 football final word cup
print(f"The 2022 football final word cups final was played the {dataset.metadata.date.date()} at {dataset.metadata.date.time() }")
print(f"score: {dataset.metadata.score}")

The 2022 football final word cups final was played the 2022-12-18 at 15:00:00
score: None


**ahaaa the final score is not provided....**

In [98]:
# let check what we can have with the teams variable
teams = dataset.metadata.teams # as precise up there we expect here a list of two variable the home and the away team
type(teams)

list

In [88]:
# what type of variable is a team
print(f"object of type: {teams[0].__class__.__name__} \nor more precisely {type(teams[0])}")

object of type: Team 
or more precisely <class 'kloppy.domain.models.common.Team'>


In [91]:
# wath are the attribute of a team ?
home_team ,away_team = teams
home_team?

Type:        Team
String form: Argentina
File:        ~/miniconda3/envs/pysport_env/lib/python3.11/site-packages/kloppy/domain/models/common.py
Docstring:  
A team in a match.

Attributes:
    team_id (str): Identifier given by the provider.
    name (str): Readable name of the team.
    ground (Ground): The team's ground (home or away).
    players (List[Player]): The team's players.
    starting_formation (FormationType, optional): The team's starting formation.

In [99]:
# here we see that we can access the team name,ID,starting formation, if the team play as home or away team and the list of player of the team.
print(f"this was a match between {home_team.name} and {away_team.name}")
print(home_team.starting_formation,away_team.starting_formation)

this was a match between Argentina and France
None None


In [ ]:
# even the starting formations are not provided

In [ ]:
# Check the list of players
home_team.players

[Player(player_id='13222', team=Team(team_id='364', name='Argentina', ground=home, starting_formation=None, formations=<TimeContainer>, players=[..., Player(player_id='8058', team=..., jersey_no=21, first_name=None, last_name=None, name='Paulo Dybala', starting=None, starting_position=<PositionType.Striker: ('Striker', 'ST', 'Attacker')>, positions=<TimeContainer>, attributes={}), Player(player_id='8025', team=..., jersey_no=3, first_name=None, last_name=None, name='Nicolas Tagliafico', starting=None, starting_position=<PositionType.LeftBack: ('Left Back', 'LB', 'FullBack')>, positions=<TimeContainer>, attributes={}), Player(player_id='1389', team=..., jersey_no=15, first_name=None, last_name=None, name='Angel Correa', starting=None, starting_position=<PositionType.Striker: ('Striker', 'ST', 'Attacker')>, positions=<TimeContainer>, attributes={}), Player(player_id='10715', team=..., jersey_no=9, first_name=None, last_name=None, name='Julian Alvarez', starting=None, starting_position=<P

In [107]:
# in a more readable way
import pandas as pd
home_players = {home_team.name+"_players" : home_team.players}
df_players = pd.DataFrame(home_players)
df_players

,Argentina_players
0,Nahuel Molina
1,Paulo Dybala
2,Nicolas Tagliafico
3,Angel Correa
4,Julian Alvarez
5,Lisandro Martinez
6,Geronimo Rulli
7,Cristian Romero
8,Emiliano Martínez
9,Guido Rodríguez


In [112]:
# which other informations can we get from the player
home_team.players[0]?

Type:        Player
String form: Nahuel Molina
File:        ~/miniconda3/envs/pysport_env/lib/python3.11/site-packages/kloppy/domain/models/common.py
Docstring:  
A player in a team.

Attributes:
    player_id (str): Identifier given by the provider.
    team (team): The player's team.
    jersey_no (int): The player's jersey number.
    first_name (str, optional): The player's first name.
    last_name (str, optional): The player's last name.
    name (str, optional): Full name of the player.
    full_name (str): If `name` is not set, this will be the concatenation
        of `first_name` and `last_name` or if these are also not set,
        the concatenation of the team's ground and the jersey number.
    starting (bool): `True` when player is part of the starting XI.
    starting_position (Position, optional): The player's starting position
        or `None` if the player is not starting.
    poisitions (TimeContainer[Position]): The player's positions over time.
    attributes (dic

In [118]:
# Let make our players dataframe more valuable with more insightfull information like the jersey number as index,
# the full name position of the player the value wich let us know if it has start the game or no at the beginig
# and its starting_position.
home_players = {home_team.name+"_players_name" : [player.full_name for player in home_team.players],
                "jersey_no": [player.jersey_no for player in home_team.players],
                "has_start_": [player.starting for player in home_team.players],
                "player_position": [player.starting_position for player in home_team.players],
                }
df_players = pd.DataFrame(home_players)

# Set index and sort in descending order
df_players = df_players.set_index("jersey_no").sort_index(ascending=True)

df_players


,Argentina_players_name,has_start_,player_position
jersey_no,,,
1,Franco Armani,None,Goalkeeper
2,Juan Foyth,None,Right Back
3,Nicolas Tagliafico,None,Left Back
4,Gonzalo Montiel,None,Right Back
5,Leandro Paredes,None,Defensive Midfield
6,German Pezzella,None,Right Center Back
7,Rodrigo de Paul,None,Center Midfield
8,Marcos Acuña,None,Left Back
9,Julian Alvarez,None,Striker


In [119]:
# let go back into another metadat the frame_rate 
dataset.metadata.frame_rate

29.97

In [ ]:
# nice the frame rate was not 25 per second but 29.97 which is strange.
what_is_this(frame_rate=int())

# compare to the time on wich the last frame were recorded
print(dataset.frames[-1].time)

This is the record of around 74.8896551724138 min of the 2022 football final world cup 
P4T19:01


In [ ]:
# this become more fishy, what is going on here ?

## remenber to ask question on it..(what represent the time attribute of a frame ?)
### Also ask question about how the tracking data time are formated.

In [120]:
# let look at coordinate_system, pitch_dimensions,and the orientation
print(dataset.metadata.coordinate_system,dataset.metadata.pitch_dimensions,dataset.metadata.orientation)

<kloppy.domain.models.common.PFFCoordinateSystem object at 0x7fca78889c90> MetricPitchDimensions(x_dim=Dimension(min=-52.5, max=52.5), y_dim=Dimension(min=-34.0, max=34.0), standardized=False, unit=<Unit.METERS: 'm'>, goal_width=7.32, goal_height=2.44, six_yard_width=18.32, six_yard_length=5.5, penalty_area_width=40.32, penalty_area_length=16.5, circle_radius=9.15, corner_radius=1, penalty_spot_distance=11, penalty_arc_radius=9.15, pitch_length=105.0, pitch_width=68.0) Orientation.HOME_AWAY


In [ ]:
# we gain no more information on coordinate_system
# we see here that the pitch was on 105m X 68m non standardized, with the different dimension of goal, six-yard and penalty_area
# and the information about the orinetation join to the definition in the documentation tell us that "The home team plays from left to right in the first period. The away team plays from left to right in the second period" 